# Init Tdx 

In [1]:
# import logging
import sys

# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd

# from JSONData import tdx_hdf5_api as h5a

# sys.stdout=stdout

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio

nest_asyncio.apply()

In [2]:
block_path = tdd.get_tdx_dir_blocknew() + '060.blk'

## 定义DEF

In [3]:
def get_sina_data_cname(df, dm, index=False):
    # index_status=False
    # dm = get_sina_data_df(code)
    if 'close' not in df.columns:
        # dd = cct.combine_dataFrame(df,dm.loc[:,['close','name']])
        dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd


def get_multi_date_duration(df, dt):
    dd = df.reset_index()
    dd = dd[dd.date >= dt]
    # dd['couts'] = dd.groupby(['code'])['code'].transform('count')
    dd = dd.set_index(['code', 'date'])
    return dd


def get_multi_code_count(df, col='code'):
    dd = df.reset_index()
    dd['couts'] = dd.groupby([col])[col].transform('count')
    dd = dd.sort_values(by=['couts', 'date'], ascending=[0, 1])
    print('count dd.couts')
    dd = dd.set_index(['code', 'date'])
    return dd


def multindex_iloc(df, index):
    label = df.index.levels[0][index]
    return df.iloc[df.index.get_loc(label)]


def get_groupby_mean_median_close(dfs):

    groupd = dfs.groupby(level=[0])

    df = groupd['close'].agg(['median', 'mean'])
    df['close'] = groupd.tail(1).reset_index().set_index(['code'])['close']
    # dfs['mean'] = groupd['close'].agg('mean')
    # dfs['median'] = groupd['close'].agg('median')

    # dfs = dfs.fillna(0)
    # idx = pd.IndexSlice
    # mask = ( (dfs['mean'] > dfs['median'])
    #         & (dfs['close'] > dfs['mean'])
    #         )
    # df=dfs.loc[idx[mask, :]]

    df = df[(df['mean'] > df['median']) & (df['close'] > df['mean'])]

    # dt_low = None
    # if dl == 1:
    #     dfs = groupd.tail(1)
    #     print("dfs tail1")
    # else:
    #     dl = 30
    #     dindex = tdd.get_tdx_Exp_day_to_df(
    #         '999999', dl=dl).sort_index(ascending=False)
    #     dt = tdd.get_duration_price_date('999999', df=dindex)
    #     dt = dindex[dindex.index >= dt].index.values
    #     dt_low = dt[-1]
    #     dtlen = len(dt) if len(dt) >0 else 1
    #     dfs = groupd.tail(dtlen)
    #     print("dfs tail:%s dt:%s"%(dtlen,dt))
    #     dfs = get_multi_date_duration(dfs,dt[-1])
    return df

def compute_df_Col_lastday(dfs,col='close',colre='perc',days=3,plus=True):
    for da in range(int(days)):
        da+=1
        if plus:
            dfs['%s%sd'%(colre,da)] = round(
                (dfs[col] - dfs[col].shift(da)) / dfs[col].shift(da) * 100, 2)
        else:
            if da == 1:
                dfs['%s%sd'%(colre,da)] = round(
                    (dfs[col] - dfs[col].shift(da)) / dfs[col].shift(da) * 100, 2)
            else:
                dfs['%s%sd'%(colre,da)] = round(
                    (dfs[col].shift(da-1) - dfs[col].shift(da)) / dfs[col].shift(da) * 100, 2)
    return dfs

In [4]:
def get_roll_mean_all(
        single=True, tdx=False, app=True, duration=100, ma_250_l=1.02,
        ma_250_h=1.11, resample='d'):
    time_s = time.time()
    if not app and cct.get_file_size(block_path) > 100 and cct.creation_date_duration(block_path) == 0:
        print("It's Today Update")
        return True
    code_list = sina_data.Sina().market('all').index.tolist()
    code_list.extend(['999999', '399001', '399006'])
    print("all code:", len(code_list))
    if duration < 300:
        h5_fname = 'tdx_all_df' + '_' + str(300)
        h5_table = 'all' + '_' + str(300)
    else:
        h5_fname = 'tdx_all_df' + '_' + str(900)
        h5_table = 'all' + '_' + str(900)
    # df = tdd.search_Tdx_multi_data_duration('tdx_all_df_300', 'all_300', df=None,code_l=code_list, start='20150501', end=None, freq=None, col=None, index='date')
    df = tdd.search_Tdx_multi_data_duration(
        h5_fname, h5_table, df=None, code_l=code_list, start=None, end=None,
        freq=None, col=None, index='date')
    # df = tdd.search_Tdx_multi_data_duration(h5_fname, h5_table, df=None,code_l=code_list, start=None, end=None, freq=None, col=None, index='date',tail=1)

    code_uniquelist = df.index.get_level_values('code').unique()

    code_select = code_uniquelist[random.randint(0, len(code_uniquelist) - 1)]
    print(round(time.time() - time_s, 2), df.index.get_level_values(
        'code').unique().shape, code_select, df.loc[code_select].shape)
    # df.groupby(level=[0]),df.index.get_level_values(0)
    # len(df.index.get_level_values('code').unique())
    # df = df[~df.index.duplicated(keep='first')]

    # multiIndex_func = {'close': 'mean', 'low': 'min', 'high': 'max', 'volume': 'sum', 'open': 'first'}
    # cct.using_Grouper(df, freq='W', col={'close': 'last'})
    # *** TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'
    # roll_dl = duration

    if resample != 'd':
        print("resample:%s" % (resample.upper()))
        # ohlc_dict = {'volume': 'sum','high': 'max','low': 'min','close': 'last'}
        # {'low': 'min', 'close':'mean'}
        # df = df.groupby(level=0).resample(resample, level=1).agg({'close':'last'})
        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(level=0).resample(resample, level=1).agg({'close':'last'})
        df = df.loc[:, ['close']].groupby(level=0).resample(
            resample, level=1).agg({'close': 'last'}).dropna()

        df = df.dropna()
        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(level=0).resample(resample, level=1, how={'low': 'min', 'close':'mean'})
        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(pd.Grouper(freq=resample)).transform(np.cumsum).resample(resample, how='ohlc')

        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(pd.TimeGrouper(resample)).transform(np.cumsum).resample(resample, how='ohlc')
        # r1 = len(df.loc['000001'])
        # r2 = len(df.loc['999999'])
        # roll_dl = r1 if r1 < r2 else r2

        # df['upper'] = map(lambda x: round((1 + 11.0 / 100) * x, 1), df.ma10d)
        # df['lower'] = map(lambda x: round((1 - 9.0 / 100) * x, 1), df.ma10d)
        # df['ene'] = map(lambda x, y: round((x + y) / 2, 1), df.upper, df.lower)
        # dfs['amount%d'%cumdays] = groupd['amount'].apply(pd.rolling_mean, cumdays)
    # df.ix[df.index.levels[0]]
    # df.ix[df.index[len(df.index)-1][0]] #last row
    # dfs = tdd.search_Tdx_multi_data_duration(df=dfs,code_l=code_list, start='20170918', end='20170918', freq=None, col=None, index='date')

    # print dfs[:1],len(dfs)
    # groupd.agg({'low': 'min'})
    # '''idx mask filter'''
    # '''
    # h5a.write_hdf_db('all300', df, table='roll200', index=False,baseCount=500, append=False, MultiIndex=MultiIndex)
    df.reset_index().sort_values(by=['code','date'],ascending=[0,1])
    print("use time:%s" % (round(time.time() - time_s, 2)))
    return df

# 读取数据

In [5]:
resample = 'd'

df = get_roll_mean_all(
    single=False, tdx=True, app=True, duration=250, ma_250_l=1.02,
    ma_250_h=1.2, resample=resample)


# df= df.groupby(level=[0]).tail(1).reset_index().set_index('code')
# df_cname = tdd.get_sina_datadf_cnamedf(df.index.tolist(), df)
# df = df.reset_index().set_index(['code', 'date'])

all code: 4959
4.74 (4940,) 300274 (299, 6)
use time:4.84


In [6]:
df.loc['688072'][-1:]

,open,high,low,close,vol,amount
date,,,,,,
2022-04-20,95.01,104.96,88.1,92.3,20563290.0,1.962506e+09


In [7]:
dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday

'2023-02-06'

## check HDF LastDate

In [8]:
df_tail1 = df.groupby(level=[0]).tail(1)
# df_tail1.index
df_tail1[df_tail1.index.get_level_values('date') == cct.get_today()].shape
df_tail1[df_tail1.index.get_level_values('date') != cct.get_today()].shape

(0, 6)

(4940, 6)

In [9]:
df_tail1[df_tail1.index.get_level_values('date') != cct.get_today()][:1]

,,open,high,low,close,vol,amount
code,date,,,,,,
999999,2021-11-11,3486.45,3534.2,3482.83,3532.79,322087957.0,4.648039e+11


In [10]:
df.info()
code_t='600519'
print(df.loc[code_t][-3:])
# df.loc['000001'][-1:]
dfbak=df.copy()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1423210 entries, ('999999', Timestamp('2023-02-06 00:00:00')) to ('603160', Timestamp('2021-11-11 00:00:00'))
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    1423210 non-null  float64
 1   high    1423210 non-null  float64
 2   low     1423210 non-null  float64
 3   close   1423210 non-null  float64
 4   vol     1423210 non-null  float64
 5   amount  1423210 non-null  float64
dtypes: float64(6)
memory usage: 70.8+ MB
               open     high      low    close        vol        amount
date                                                                   
2021-11-15  1730.10  1740.41  1706.41  1717.55  2494735.0  4.410975e+09
2021-11-12  1734.41  1741.46  1723.07  1730.19  1760663.0  3.124401e+09
2021-11-11  1709.35  1726.01  1697.91  1726.01  2265187.0  3.987243e+09


In [16]:
dfbak = dfbak.reset_index().set_index('code').sort_values(by='date',ascending=1).reset_index().set_index(['code','date'])

## 初始化参数

In [17]:
app = True
tdx = True
duration = 300
ma_250_l = 1.02
ma_250_h = 1.2


time_s = time.time()
dfs = dfbak.copy()
# dfs = df.loc['600519']
# groupd = dfs.groupby(level=[0])
# groupd.['close']
# rollma = ['5','10','60','100','200']
# rollma = ['5','10','250']
# df.index.get_level_values('code')[0]
if resample.upper() == 'D' or resample.lower() == 'd':
    if duration < 300:
        rollma = ['10', '5', '26', '200']
    else:
        rollma = ['10', '5', '26', '250']

elif resample.upper() == 'W' or resample.lower() == 'w':
    rollma = ['5', '10']
else:
    rollma = ['5', '10']

# rollma.extend([str(duration)])

# import ipdb;ipdb.set_trace()
# df.loc['300130'][:2]

# dfs['mean'] = groupd['close'].agg('mean')
# dfs['median'] = groupd['close'].agg('median')

# BOLL:MA(CV,N), COLORMAGENTA;
# UPPER:BOLL+P*STD(CV,N), COLORYELLOW;
# LOWER:BOLL-P*STD(CV,N), COLORYELLOW;
# df['BollUpper'] = df.close.shift(10).mean()+2*df.close.shift(10).std()
# df['BollDown'] = df.close.shift(10).mean()-2*df.close.shift(10).std()
for da in rollma:
    cumdays = int(da)
    dfs['ma%d' % cumdays] = round(dfs['close'].rolling(cumdays).mean(), 2)

    # dfs['upper'] = dfs['ma%d' % cumdays].apply(
    #     lambda x: round((1 + 11.0 / 100) * x, 1))
    # dfs['lower'] = dfs['ma%d' % cumdays].apply(
    #     lambda x: round((1 - 9.0 / 100) * x, 1))
    # dfs['boll'] = list(map(lambda x, y: round(
    #     (x + y) / 2, 1), dfs['upper'], dfs['lower']))
dfs['upper'] = round(
    dfs['close'].rolling(20).mean() + 2 *
    dfs['close'].rolling(20).std(),
    2)
dfs['lower'] = round(
    dfs['close'].rolling(20).mean() - 2 *
    dfs['close'].rolling(20).std(),
    2)
dfs['boll'] = round(dfs['close'].rolling(20).mean(), 2)

dfs=compute_df_Col_lastday(dfs,col='close',colre='perc',days=5)
dfs=compute_df_Col_lastday(dfs,col='vol',colre='vol',days=5,plus=False)

# dfs['perc1d'] = round(
#     (dfs['close'] - dfs['close'].shift(1)) / dfs['close'].shift(1) * 100, 2)
# dfs['perc3d'] = round(
#     (dfs['close'] - dfs['close'].shift(3)) / dfs['close'].shift(3) * 100, 2)
# dfs['perc5d'] = round(
#     (dfs['close'] - dfs['close'].shift(5)) / dfs['close'].shift(3) * 100, 2)



In [20]:
# dfs.loc['301066']
dfs['ma51d'] = round(dfs['close'].rolling(5).mean(), 2)
print(dfs.loc['999999'])
# print(dfs.loc['600519'])

               open     high      low    close          vol        amount  \
date                                                                        
2021-11-11  3486.45  3534.20  3482.83  3532.79  322087957.0  4.648039e+11   
2021-11-12  3534.15  3543.65  3527.39  3539.10  308683354.0  4.433984e+11   
2021-11-15  3542.90  3550.44  3521.29  3533.30  313017612.0  4.640654e+11   
2021-11-16  3530.46  3549.77  3517.81  3521.79  310399909.0  4.447008e+11   
2021-11-17  3518.56  3537.51  3513.52  3537.37  276735671.0  4.015523e+11   
...             ...      ...      ...      ...          ...           ...   
2023-01-31  3266.14  3277.25  3252.01  3255.67  300162957.0  3.752607e+11   
2023-02-01  3262.20  3284.92  3245.41  3284.92  339509410.0  4.133952e+11   
2023-02-02  3291.78  3295.76  3272.41  3285.67  333413133.0  4.286628e+11   
2023-02-03  3275.66  3275.66  3235.35  3263.41  293342814.0  3.786485e+11   
2023-02-06  3244.70  3250.35  3226.21  3238.70  267053107.0  3.491273e+11   

In [13]:
dfs = dfs.dropna()
single = False
dt_low = None
df_idx = None
if single:
    dfs_dtlen = groupd.tail(1)
    print("dfs tail1")
else:
    single = True
    dl = 30
    dindex = tdd.get_tdx_Exp_day_to_df(
        '999999', dl=dl, resample=resample).sort_index(ascending=False)
    dt = tdd.get_duration_price_date('999999', df=dindex)

    dt = dindex[dindex.index >= dt].index.values
    dt_low = dt[-1]
    print('dt_low:%s' % (dt_low))
    dtlen = len(dt) if len(dt) > 0 else 1
    dfs_dtlen = groupd.tail(dtlen)
    # import ipdb;ipdb.set_trace()
    df_idx = get_groupby_mean_median_close(dfs_dtlen)

    print(("dfs tail:%s dt:%s" % (dtlen, dt)))
    dfs_dtlen = get_multi_date_duration(dfs_dtlen, dt[-1])

# groupd2 = dfs.groupby(level=[0])
# dfs['ma%d'%cumdays] = groupd['close'].apply(pd.rolling_mean, cumdays)

# dfs.reset_index().groupby(['code'])['date'].transform('count')
# single = True

print("Low:%s" % (dt[-1]))
dfs_dtlen[:5]
dfs_dtlen[('ma%s') % (rollma[-1])][:2]
print(df_idx)

dt_low:2022-12-23
dfs tail:27 dt:['2023-02-07' '2023-02-06' '2023-02-03' '2023-02-02' '2023-02-01'
 '2023-01-31' '2023-01-30' '2023-01-20' '2023-01-19' '2023-01-18'
 '2023-01-17' '2023-01-16' '2023-01-13' '2023-01-12' '2023-01-11'
 '2023-01-10' '2023-01-09' '2023-01-06' '2023-01-05' '2023-01-04'
 '2023-01-03' '2022-12-30' '2022-12-29' '2022-12-28' '2022-12-27'
 '2022-12-26' '2022-12-23']
Low:2022-12-23


open     high      low    close          vol  \
code   date                                                          
999999 2022-12-23  3038.84  3061.87  3031.54  3045.87  192020305.0   
       2022-12-26  3048.20  3071.84  3047.35  3065.56  206503893.0   
       2022-12-27  3077.75  3098.08  3074.31  3095.57  222218322.0   
       2022-12-28  3088.62  3098.65  3079.43  3087.40  224554151.0   
       2022-12-29  3076.73  3086.00  3064.46  3073.70  215570676.0   

                         amount     ma10      ma5     ma26    ma250  ...  \
code   date                                                          ...   
999999 2022-12-23  2.333352e+11  3121.80  3069.92  3134.87  3249.33  ...   
       2022-12-26  2.554703e+11  3110.45  3061.61  3133.65  3247.00  ...   
       2022-12-27  2.679468e+11  3102.38  3065.97  3134.06  3244.69  ...   
       2022-12-28  2.605085e+11  3093.46  3069.77  3134.00  3242.51  ...   
       2022-12-29  2.539170e+11  3083.97  3073.62  3133.11  3240.43  ...   

                   perc2d  perc3d  perc4d  perc5d  vol1d  vol2d  vol3d  vol4d  \
code   date                                                                     
999999 2022-12-23   -0.73   -0.91   -1.97   -3.85 -14.14  18.29 -13.60 -21.43   
       2022-12-26    0.36   -0.09   -0.27   -1.34   7.54 -14.14  18.29 -13.60   
       2022-12-27    1.63    1.35    0.89    0.71   7.61   7.54 -14.14  18.29   
       2022-12-28    0.71    1.36    1.08    0.62   1.05   7.61   7.54 -14.14   
       2022-12-29   -0.71    0.27    0.91    0.63  -4.00   1.05   7.61   7.54   

                   vol5d    ma51d  
code   date                        
999999 2022-12-23   7.21  3069.92  
       2022-12-26 -21.43  3061.61  
       2022-12-27 -13.60  3065.97  
       2022-12-28  18.29  3069.77  
       2022-12-29 -14.14  3073.62  

[5 rows x 24 columns]

code    date      
999999  2022-12-23    3249.33
        2022-12-26    3247.00
Name: ma250, dtype: float64

         median         mean     close
code                                  
000004     9.70     9.808519    10.380
000008     2.38     2.387407     2.480
000009    12.22    12.231481    12.580
000012     6.88     6.923333     7.140
000016     4.63     4.673333     4.790
...         ...          ...       ...
688789   169.80   175.442222   183.520
688798   105.45   108.826296   117.800
688799    33.38    33.571111    34.700
688981    41.86    41.927037    43.180
999999  3176.08  3181.613111  3248.094

[2938 rows x 3 columns]


## 基础处理

In [14]:
time_s = time.time()


def get_mask_df(dfs):
    dfs = dfs.fillna(0)
    idx = pd.IndexSlice
    # mask = (dfs[('ma%s')%(rollma[0])] > dfs[('ma%s')%(rollma[1])]) & (dfs[('ma%s')%(rollma[-1])] > 0) & (dfs[('close')] > dfs[('ma%s')%(rollma[0])])  & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])])
    # mask = (dfs[('ma%s')%(rollma[0])] > dfs[('ma%s')%(rollma[1])]) & (dfs[('ma%s')%(rollma[-1])] > 0) & (dfs[('close')] > dfs[('ma%s')%(rollma[1])])  & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])])
    # mask = (dfs[('ma%s')%(rollma[0])] > dfs[('ma%s')%(rollma[1])]) & (dfs[('ma%s')%(rollma[-1])] > 0) &  (dfs[('close')] > dfs[('ma%s')%(rollma[-1])])

    # mask = ( (dfs[('ma%s')%(rollma[0])] > 0) & (dfs[('ma%s')%(rollma[-1])] > 0) & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])]) & (dfs[('close')] > dfs[('ma%s')%(rollma[0])]))
    # mask = ( (dfs[('ma%s')%(rollma[0])] > 0) & (dfs[('ma%s')%(rollma[-1])] > 0)
    #         & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])]*ma_250_l)
    #         & (dfs[('close')] < dfs[('ma%s')%(rollma[-1])]*ma_250_h)
    #         & (dfs[('close')] > dfs[('ma%s')%(rollma[0])]))

    # & (dfs['mean'] > dfs['median'])
    # & (dfs['close'] > dfs['mean'])

    if len(rollma) > 1:

        mask = ((dfs[('ma%s') % (rollma[0])] > 0) & (dfs[('ma%s') % (rollma[-1])] > 0) &
                (dfs[('ma%s') % (rollma[0])] > dfs[('ma%s') % (rollma[-1])]) &
                (dfs[('close')] > dfs[('ma%s') % (rollma[0])]) &
                (dfs[('close')] > dfs[('ma%s') % (rollma[-1])]*ma_250_h) &
                ((dfs[('close')] > dfs['boll']) | (dfs[('close')] > dfs['upper']))
                )
    else:
        mask = ((dfs[('ma%s') % (rollma[0])] > 0) &
                (dfs[('close')] > dfs[('ma%s')%(rollma[0])]) &
                ((dfs[('close')] > dfs['boll']) | (dfs[('close')] > dfs['upper']) )
                )

    return dfs.loc[idx[mask, :]]

# mask = ((dfs[('close')] > dfs[('ma%s')%(rollma[-1])]))


df_mask = get_mask_df(dfs_dtlen)
print(df_mask.shape)
df_mask = get_multi_code_count(df_mask)
print((df_mask.couts[:5], df_mask.shape))

# import ipdb;ipdb.set_trace()
# df.sort_values(by='couts',ascending=0)
# groupd.first()[:2],groupd.last()[:2]
# groupd = df250.groupby(level=[0])
# '''
# groupd.transform(lambda x: x.iloc[-1])
# groupd.last()
# groupd.apply(lambda x: x.close > x.ma250)
# df.shape,df.sort_index(ascending=False)[:5]
# ?groupd.agg
# groupd = df.groupby(level=[0])
# groupd['close'].apply(pd.rolling_mean, 250, min_periods=1)
# ex:# Group df by df.platoon, then apply a rolling mean lambda function to df.casualties
# df.groupby('Platoon')['Casualties'].apply(lambda x:x.rolling(center=False,window=2).mean())

code_uniquelist = df_mask.index.get_level_values('code').unique()
code_select = code_uniquelist[random.randint(0, len(code_uniquelist) - 1)]

# if app:
#     print(round(time.time() - time_s, 2), 's', df.index.get_level_values(
#         'code').unique().shape, code_select, df.loc[code_select][-1:])

if single:
    # groupd = df.groupby(level=[0])
    if tdx:
        # block_path = tdd.get_tdx_dir_blocknew() + '060.blk'
        # if cct.get_work_time():
        #     codew = df[df.date == cct.get_today()].index.tolist()

        if dt_low is not None:

            groupd2 = df_mask.groupby(level=[0])
            df_mask = groupd2.tail(1)
            df_mask = df_mask.reset_index().set_index('code')
            # import ipdb;ipdb.set_trace()

            # df = df[(df.date >= dt_low) & (df.date <= cct.get_today())]
            dd = df_mask[(df_mask.date == dt_low)]
            # df = df[(df_mask.date >= cct.last_tddate(1))]
            df_mask = df_mask[(df_mask.date >= df_mask.date.max())]
            # import ipdb;ipdb.set_trace()
            print(("df:%s %s df_idx:%s" %
                  (len(df_mask), df_mask.index[:5], len(df_idx))))

            if df_idx is not None and len(df_mask) > 0 and len(df_idx) > 0:
                idx_set_ = [x for x in df_idx.index if x in df_mask.index]
                df_mask = df_mask.loc[idx_set_, :].dropna()
            print(("Main Down dd :%s MainUP df:%s couts std:%0.1f " %
                  (len(dd), len(df_mask), df_mask.couts.std())))
            # print df.date.mode()[0]
            df_mask = df_mask.sort_values(by='couts', ascending=1)
            df_mask = df_mask[df_mask.couts > df_mask.couts.std()]
            # df = df[(df.date >= df.date.mode()[0]) & (df.date <= cct.get_today())]
            codew = df_mask.index.tolist()

            if app:
                print(
                    "time: %s %s %s" %
                    (round(time.time() - time_s, 2),
                     'groupd2', len(df_mask)))

        else:
            df_mask = df_mask.reset_index().set_index('code')
            df_mask = df_mask[(df_mask.date >= cct.last_tddate(days=10)) &
                              (df_mask.date <= cct.get_today())]
            codew = df_mask.index.tolist()

        top_temp = tdd.get_sina_datadf_cnamedf(codew, df_mask)
        top_temp = top_temp[(~top_temp.index.str.contains(
            '688')) & (~top_temp.name.str.contains('ST'))]
        top_temp = top_temp.sort_values(by=['couts'], ascending=[0])
        codew = top_temp.index.tolist()

        # clean st and 688

    # df['date'] = df['date'].apply(lambda x:(x.replace('-','')))
    # df['date'] = df['date'].astype(int)
    # print df_mask.loc[code_select].T,df_mask.shape
#     MultiIndex = False
# else:
#     MultiIndex = True

(10611, 24)
count dd.couts
(code    date      
301239  2022-12-23    27
        2022-12-26    27
        2022-12-27    27
        2022-12-28    27
        2022-12-29    27
Name: couts, dtype: int64, (10611, 25))
df:564 Index(['301239', '603255', '301267', '301333', '300585'], dtype='object', name='code') df_idx:2938
Main Down dd :4 MainUP df:231 couts std:6.6 
time: 0.04 groupd2 66


In [15]:
dfs_dtlen.loc['002987'][:1]

,open,high,low,close,vol,amount,ma10,ma5,ma26,ma250,...,perc2d,perc3d,perc4d,perc5d,vol1d,vol2d,vol3d,vol4d,vol5d,ma51d
date,,,,,,,,,,,,,,,,,,,,,
2022-12-23,25.12,25.92,25.07,25.6,4349418.0,111461984.0,26.5,25.83,28.62,24.24,...,-1.39,-1.2,-2.59,-3.03,-5.55,7.79,-24.84,10.06,-28.81,25.83


# 最近30天筛选

In [16]:
# 截取最后5天数据
dfs_tail = dfs.groupby(level=[0]).tail(30)
# 还原多索引
# dfs_tail = dfs_tail.reset_index().set_index(['code','date'])
dfs_tail.loc['002987'][:1]

,open,high,low,close,vol,amount,ma10,ma5,ma26,ma250,...,perc2d,perc3d,perc4d,perc5d,vol1d,vol2d,vol3d,vol4d,vol5d,ma51d
date,,,,,,,,,,,,,,,,,,,,,
2022-12-20,26.87,26.96,25.66,25.91,5684001.0,148359552.0,27.62,26.59,29.4,24.19,...,-1.86,-5.37,-3.97,-3.46,10.06,-28.81,13.18,8.86,-45.41,26.59


In [17]:
mask250 = ((dfs_tail[('ma%s') % (rollma[0])] > 0) & (dfs_tail[('ma%s') % (rollma[-1])] > 0) &
           (dfs_tail[('close')] > dfs_tail[('ma%s') % (rollma[0])]) &
            (dfs_tail[('close')] > dfs_tail[('ma%s') % (rollma[-1])]*1.03) &
              (dfs_tail[('close')] < dfs_tail[('ma%s') % (rollma[-1])]*1.1) &
                (dfs_tail[('close')] > dfs_tail[('ma26')]) &
           (dfs_tail[('close')] < dfs_tail[('ma26'   )]*1.1))
idx = pd.IndexSlice
df_mask250 = dfs_tail.loc[idx[mask250, :]]
print(df_mask250.shape)
df_mask250 = get_multi_code_count(df_mask250)
print((df_mask250.couts[:1], df_mask250.shape))

groupd250 = df_mask250.groupby(level=[0])
df_mask250 = groupd250.tail(1)
print((df_mask250.couts[:5], df_mask250.shape))
print(df_mask.loc['002987'])

(8680, 24)
count dd.couts
(code    date      
300849  2023-01-03    20
Name: couts, dtype: int64, (8680, 25))
(code    date      
300849  2023-02-06    20
601988  2023-01-31    19
600674  2023-02-02    18
600729  2023-02-03    18
688009  2023-02-06    18
Name: couts, dtype: int64, (1604, 25))


KeyError: '002987'

# 最近9天数据

In [18]:
dfs_tail9 = dfs.groupby(level=[0]).tail(5)
# 还原多索引
# dfs_tail = dfs_tail.reset_index().set_index(['code','date'])
dfs_tail9.loc['002987'][:1]

,open,high,low,close,vol,amount,ma10,ma5,ma26,ma250,...,perc2d,perc3d,perc4d,perc5d,vol1d,vol2d,vol3d,vol4d,vol5d,ma51d
date,,,,,,,,,,,,,,,,,,,,,
2023-02-01,28.88,30.16,28.87,30.04,8889888.0,263173696.0,28.51,29.22,27.52,24.62,...,1.87,3.8,4.41,6.3,38.46,-40.85,28.67,-16.57,-18.06,29.22


## 超跌反弹年线大于MA26

In [27]:
mask250 = ((dfs_tail9[('close')] > dfs_tail9.upper) &
           (dfs_tail9[('close')] > dfs_tail9[('ma5')]) &
           (dfs_tail9[('ma5')] > dfs_tail9[('ma26')])
           )
# & (dfs_tail9[('close')] < dfs_tail9[('ma26'  )]*1.1))
idx = pd.IndexSlice
df_mask250 = dfs_tail9.loc[idx[mask250, :]]
df_mask250 = get_multi_code_count(df_mask250)
# print((df_mask250.couts[:1], df_mask250.shape))

groupd250 = df_mask250.groupby(level=[0])
df_mask250 = groupd250.tail(1)
df_mask250 = groupd250.tail(1).reset_index().set_index('code')
df_mask250 = get_sina_data_cname(df_mask250, dm)

df_masklastd = df_mask250[(df_mask250.date >= lastday)
                          & (df_mask250.couts == 1)]

print("src:%s  lastd:%s " % (df_mask250.shape, df_masklastd.shape))


code_t = '300816'
code_t = '600519'


df_masklastd = df_masklastd.sort_values(
    by=['perc1d', 'perc5d'],
    ascending=[0, 1])

print(
    (df_masklastd.loc
     [:, ['name', 'couts', 'date', 'perc1d', 'perc3d', 'perc5d']][: 5],
     df_masklastd.shape,))

df_masklastd.loc[code_t, ['name', 'couts',
                          'date', 'perc1d', 'perc3d', 'perc5d']]

df_mask250 = groupd250.head(1)

df_masklastd.loc[code_t]
# print(dfs_tail9.loc[code_t])

count dd.couts
src:(2310, 27)  lastd:(304, 27) 
(        name  couts       date    perc1d    perc3d    perc5d
code                                                        
600519  贵州茅台      1 2023-02-07  27934.32   1732.93  13421.44
999999   NaN      1 2023-02-07  25098.56  24808.70  24732.52
600436   片仔癀      1 2023-02-07   9416.03   4344.61   1971.88
300896   爱美客      1 2023-02-07   9186.74   1767.49   1353.74
600600  青岛啤酒      1 2023-02-07   8865.79    932.42    559.42, (304, 27))


name                     贵州茅台
couts                       1
date      2023-02-07 00:00:00
perc1d               27934.32
perc3d                1732.93
perc5d               13421.44
Name: 600519, dtype: object

date      2023-02-07 00:00:00
open                  1808.08
high                   1808.8
low                   1787.73
close                  1797.0
vol                 2432171.0
amount           4373671735.0
ma10                    199.5
ma5                    384.94
ma26                    86.45
ma250                   24.78
upper                  903.54
lower                 -689.76
boll                   106.89
perc1d               27934.32
perc2d               42890.43
perc3d                1732.93
perc4d                9323.18
perc5d               13421.44
vol1d                  -28.47
vol2d                  -90.66
vol3d                  368.41
vol4d                 1189.97
vol5d                  -97.94
ma51d                  384.94
couts                       1
name                     贵州茅台
Name: 600519, dtype: object

               open    high      low    close        vol        amount  \
date                                                                     
2023-02-01  1854.98  1859.0  1811.40  1844.97  3397389.0  6.244679e+09   
2023-02-02  1848.38  1859.0  1826.00  1836.11  2975886.0  5.465509e+09   
2023-02-03  1820.00  1826.0  1795.68  1818.00  3494490.0  6.324635e+09   
2023-02-06  1780.00  1795.0  1760.00  1795.00  4266095.0  7.595739e+09   
2023-02-07  1808.08  1808.8  1787.73  1797.00  2432171.0  4.373672e+09   

               ma10      ma5     ma26    ma250  ...    perc2d   perc3d  \
date                                            ...                      
2023-02-01  1875.39  1863.79  1803.53  1765.94  ...     -2.28    -0.81   
2023-02-02  1875.60  1854.97  1809.03  1766.01  ...     -0.52    -2.75   
2023-02-03  1868.70  1846.57  1812.91  1765.89  ...     -1.46    -1.50   
2023-02-06  1856.91  1827.97  1814.79  1765.60  ...     -2.24    -2.71   
2023-02-07   199.50   384.94    86.45

In [29]:
print(dfs.loc[code_t])

               open     high      low    close        vol        amount  \
date                                                                      
2021-11-10  1746.42  1751.41  1691.41  1710.40  3534877.0  6.204803e+09   
2021-11-11  1709.35  1726.01  1697.91  1726.01  2265187.0  3.987243e+09   
2021-11-12  1734.41  1741.46  1723.07  1730.19  1760663.0  3.124401e+09   
2021-11-15  1730.10  1740.41  1706.41  1717.55  2494735.0  4.410975e+09   
2021-11-16  1724.41  1774.41  1721.41  1764.41  3352624.0  6.033537e+09   
...             ...      ...      ...      ...        ...           ...   
2023-02-01  1854.98  1859.00  1811.40  1844.97  3397389.0  6.244679e+09   
2023-02-02  1848.38  1859.00  1826.00  1836.11  2975886.0  5.465509e+09   
2023-02-03  1820.00  1826.00  1795.68  1818.00  3494490.0  6.324635e+09   
2023-02-06  1780.00  1795.00  1760.00  1795.00  4266095.0  7.595739e+09   
2023-02-07  1808.08  1808.80  1787.73  1797.00  2432171.0  4.373672e+09   

               ma10     

# 默认年线筛选

In [52]:
top_temp.shape
top_temp.index

(65, 17)

Index(['300996', '300354', '003007', '301289', '601958', '002401', '002847',
       '301306', '300550', '002981', '600933', '301316', '001228', '301311',
       '002458', '600962', '000729', '600476', '301369', '301290', '603298',
       '301335', '301300', '603235', '002862', '003031', '002908', '603182',
       '000423', '301129', '300634', '301269', '603757', '301197', '301160',
       '000695', '600845', '600633', '002777', '002209', '002077', '000539',
       '603823', '000933', '603927', '000026', '002747', '300835', '002952',
       '002836', '603029', '002219', '603516', '002155', '002384', '002686',
       '603950', '002800', '300415', '300161', '300275', '300427', '300645',
       '603901', '603130'],
      dtype='object', name='code')

# 写入060

In [53]:
if app:
    print("Write blk:%s" % (block_path))
    hdf5_wri = cct.cct_raw_input("rewrite code [Y] or append [N]:")
    if len(hdf5_wri) == 0:
        pass
    elif hdf5_wri == 'y' or hdf5_wri == 'Y':
        append_status = False
    else:
        append_status = True
else:
    append_status = False

if hdf5_wri == 'y' and len(codew) > 10:
    cct.write_to_blocknew(
        block_path, codew, append_status, doubleFile=False, keep_last=0)
    print("write:%s block_path:%s" % (len(codew), block_path))
else:
    print("write error:%s block_path:%s" % (len(codew), block_path))

Write blk:D:\MacTools\WinTools\new_tdx\T0002\blocknew\060.blk


rewrite code [Y] or append [N]: n


write error:65 block_path:D:\MacTools\WinTools\new_tdx\T0002\blocknew\060.blk
